In [1]:
import kagglehub

# Download the dataset (replace "abhinavmoudgil95/short-jokes" with the desired dataset name)
path = kagglehub.dataset_download("abhinavmoudgil95/short-jokes")

# Print the path where the dataset is saved
print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/short-jokes


In [2]:
import pandas as pd

# Load the dataset (replace the file name with the actual CSV file in the dataset)
jokes_data = pd.read_csv(path + "/shortjokes.csv")

# Display the first few rows of the dataset
print(jokes_data.head())


   ID                                               Joke
0   1  [me narrating a documentary about narrators] "...
1   2  Telling my daughter garlic is good for you. Go...
2   3  I've been going through a really rough period ...
3   4  If I could have dinner with anyone, dead or al...
4   5     Two guys walk into a bar. The third guy ducks.


In [3]:
# Check the structure of the dataset
print(jokes_data.info())

# Display the first few rows
print(jokes_data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231657 entries, 0 to 231656
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      231657 non-null  int64 
 1   Joke    231657 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.5+ MB
None
   ID                                               Joke
0   1  [me narrating a documentary about narrators] "...
1   2  Telling my daughter garlic is good for you. Go...
2   3  I've been going through a really rough period ...
3   4  If I could have dinner with anyone, dead or al...
4   5     Two guys walk into a bar. The third guy ducks.


In [4]:
jokes_data = jokes_data.sample(n=5000, random_state=42)

In [62]:
jokes_data.loc[4510]

ID                                                   4511
Joke    What do all battered women have in common? The...
Name: 4510, dtype: object

In [6]:
jokes_data['Joke'] = jokes_data['Joke'].str.strip()


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping


In [8]:
tokenizer = Tokenizer()

In [53]:
tokenizer.fit_on_texts(jokes_data['Joke'])

In [54]:
len(token.word_index)

10485

In [14]:
input_sequences=[]
for sentence in jokes_data['Joke']:
    tokenized_sentence = token.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])


In [43]:
max_len = max([len(x) for x in input_sequences])
max_len

44

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [21]:
X = padded_input_sequences[:,:-1]

In [22]:
y = padded_input_sequences[:,-1]

In [23]:
X.shape

(83483, 43)

In [24]:
y.shape

(83483,)

In [25]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=10486)

In [26]:
y.shape

(83483, 10486)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [28]:
model = Sequential()
model.add(Embedding(10486, 100, input_length=43))
model.add(LSTM(150))
model.add(Dense(10486, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
history = model.fit(
    X, y,
    epochs=100,
    batch_size=64,
    #validation_split=0.2,  # Automatically split 20% of data for validation
    #callbacks=[early_stopping],
    verbose=1
)


Epoch 1/100
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.0410 - loss: 7.3786 - val_accuracy: 0.0576 - val_loss: 6.9356
Epoch 2/100
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.0683 - loss: 6.5368 - val_accuracy: 0.0937 - val_loss: 6.7063
Epoch 3/100
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.1014 - loss: 6.0368 - val_accuracy: 0.1105 - val_loss: 6.6315
Epoch 4/100
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.1295 - loss: 5.6456 - val_accuracy: 0.1253 - val_loss: 6.6541
Epoch 5/100
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.1518 - loss: 5.2926 - val_accuracy: 0.1318 - val_loss: 6.6895
Epoch 6/100
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.1723 - loss: 4.9621 - val_accuracy: 0.1349 - val_loss: 6.7664
Epoch 7/100
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.1910 - loss: 4.6454 - val_accuracy: 0.1362 - val_loss: 6.8718
Epoch 8/100
1044/1044 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 

In [68]:
import numpy as np
import time
text = "women"

for i in range(30):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=44, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
women are
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
women are like
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
women are like gremlins
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
women are like gremlins get
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
women are like gremlins get them
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
women are like gremlins get them wet
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
women are like gremlins get them wet and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
women are like gremlins get them wet and they
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
women are like gremlins get them wet and they get
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
women are like gremlins get them wet and they get into
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
women are like gremlins get them wet and they get into all
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
women are like gremlins get them wet and they get into all sorts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
women are like gremlins get them wet and th